Install dependencies with:
```sh
pip install -r requirements.txt
```

For some functionality you need a valid API-key.  
Export as environment variables.
Afterwards run notebook from same shell.


Bash/Sh/Zsh:
```sh
export BINANCE_API_KEY='your_api_key'
export BINANCE_API_SECRET='your_secret_key'

# Check
echo $BINANCE_API_KEY
echo $BINANCE_API_SECRET
```
Powershell:
```powershell
$env:BINANCE_API_KEY = 'your_api_key'
$env:BINANCE_API_SECRET = 'your_secret_key'

# Check
$env:BINANCE_API_KEY
$env:BINANCE_API_KEY
```

In [1]:
import os
import pandas as pd
import numpy as np
from binance.spot import Spot as Client

In [2]:
# Ommit base_url argument to default to api.binance.com
client = Client(os.getenv('BINANCE_API_KEY'), os.getenv('BINANCE_API_SECRET'), base_url='https://testnet.binance.vision')
print(client.time())

{'serverTime': 1651944820012}


In [3]:
# print(client.account()['balances'])
pd.json_normalize(client.account()['balances'])
# client.historical_trades()
# asset = client.klines('BTCUSDT', '1m')

,asset,free,locked
0,BNB,1000.00000000,0.00000000
1,BTC,1.00000000,0.00000000
2,BUSD,10000.00000000,0.00000000
3,ETH,100.00000000,0.00000000
4,LTC,500.00000000,0.00000000
5,TRX,500000.00000000,0.00000000
6,USDT,10000.00000000,0.00000000
7,XRP,50000.00000000,0.00000000


In [97]:
assets = []
symbols = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'LTCUSDT', 'TRXUSDT', 'XRPUSDT' ]

In [98]:
for symbol in symbols:
    assets.append(client.klines(symbol, "1d"))

In [103]:
# for i in len(assets):
[c[4] for c in assets]

In [96]:
assets[1]

[[1651622400000,
  '38127.16000000',
  '67952.01000000',
  '9000.00000000',
  '39695.80000000',
  '3289.47888900',
  1651708799999,
  '128626868.06394152',
  109403,
  '2070.21409400',
  '80943515.80085649',
  '0'],
 [1651708800000,
  '39690.01000000',
  '48256.09000000',
  '12000.00000000',
  '36551.04000000',
  '3747.85911700',
  1651795199999,
  '142179013.42520536',
  136426,
  '2305.80703300',
  '87220355.25958719',
  '0'],
 [1651795200000,
  '36552.97000000',
  '90000.00000000',
  '9500.00000000',
  '35844.54000000',
  '5133.87878000',
  1651881599999,
  '185618082.67083564',
  195320,
  '2791.12783400',
  '100904636.74885754',
  '0'],
 [1651881600000,
  '35851.70000000',
  '50000.00000000',
  '2000.00000000',
  '35952.53000000',
  '1606.43292700',
  1651967999999,
  '57714392.76805852',
  70321,
  '851.83485200',
  '30608434.65229552',
  '0']]

In [79]:

client.exchange_info()['symbols']

[{'symbol': 'BNBBUSD',
  'status': 'TRADING',
  'baseAsset': 'BNB',
  'baseAssetPrecision': 8,
  'quoteAsset': 'BUSD',
  'quotePrecision': 8,
  'quoteAssetPrecision': 8,
  'baseCommissionPrecision': 8,
  'quoteCommissionPrecision': 8,
  'orderTypes': ['LIMIT',
   'LIMIT_MAKER',
   'MARKET',
   'STOP_LOSS_LIMIT',
   'TAKE_PROFIT_LIMIT'],
  'icebergAllowed': True,
  'ocoAllowed': True,
  'quoteOrderQtyMarketAllowed': True,
  'allowTrailingStop': True,
  'isSpotTradingAllowed': True,
  'isMarginTradingAllowed': False,
  'filters': [{'filterType': 'PRICE_FILTER',
    'minPrice': '0.01000000',
    'maxPrice': '10000.00000000',
    'tickSize': '0.01000000'},
   {'filterType': 'PERCENT_PRICE',
    'multiplierUp': '5',
    'multiplierDown': '0.2',
    'avgPriceMins': 5},
   {'filterType': 'LOT_SIZE',
    'minQty': '0.01000000',
    'maxQty': '9000.00000000',
    'stepSize': '0.01000000'},
   {'filterType': 'MIN_NOTIONAL',
    'minNotional': '10.00000000',
    'applyToMarket': True,
    'avgPri